In [ ]:
from stable_baselines3.common.env_util import make_atari_env
import time

In [ ]:
env = make_atari_env("BreakoutNoFrameskip-v4", seed = 0,monitor_dir='./logs/')

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
import os

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self,check_freq,save_path, verbose=1):
        super(TrainAndLoggingCallback,self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path,'best_model_ppo_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'

callback = TrainAndLoggingCallback(check_freq=10000, save_path = CHECKPOINT_DIR)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack

In [ ]:
#Empile 4 images consécutives
env = VecFrameStack(env, 4)

#### Charger Modèle et continuer entrainement

In [ ]:
#Charger modele
model = PPO.load(os.path.join(CHECKPOINT_DIR, 'last_save_ppo_breakout.zip'), env=env)

In [ ]:
model.learn(total_timesteps = 10000000, callback = callback,reset_num_timesteps=False,tb_log_name = 'Breakout_PPO_V2')